# 문제 1: 개와 고양이 분류

- **데이터셋**: Kaggle Cats and Dogs (128×128 RGB)
- **검증**: 반복 홀드아웃 5회, 훈련:검증 = 3:2
- **평가**: Accuracy, F1 Score (Micro)

## 모델 구조
1. **LeNet-5** (베이스라인) - 수업시간에 배운 고전적 CNN
2. **VGGNet-style** - 작은 3×3 커널을 깊게 쌓은 구조
3. **ResNetCNN** - Residual Connection으로 깊은 학습
4. **SEResNet** (최고 성능 예상) - SE attention + Residual

## 1. 라이브러리

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile

device = torch.device('cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu')
print(f"Device: {device}")

In [ ]:
def set_seed(seed):
    torch.manual_seed(seed)
    np.random.seed(seed)

## 2. 데이터셋 다운로드

Kaggle API 필요: `pip install kaggle`

kaggle.json을 ~/.kaggle/에 저장해야 함

In [ ]:
# 데이터 디렉토리
DATA_DIR = './data/cats_dogs'
os.makedirs(DATA_DIR, exist_ok=True)

In [ ]:
# Kaggle에서 다운로드
!kaggle datasets download -d samuelcortinhas/cats-and-dogs-image-classification -p {DATA_DIR}

In [ ]:
# 압축 해제
zip_path = os.path.join(DATA_DIR, 'cats-and-dogs-image-classification.zip')
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(DATA_DIR)
print("압축 해제 완료")

## 3. 데이터 준비

In [ ]:
class CatsDogsDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.transform = transform
        self.image_paths = []
        self.labels = []
        
        for class_name, class_idx in [('cats', 0), ('dogs', 1)]:
            class_dir = os.path.join(data_dir, class_name)
            if os.path.exists(class_dir):
                for img_name in os.listdir(class_dir):
                    if img_name.endswith(('.jpg', '.jpeg', '.png')):
                        self.image_paths.append(os.path.join(class_dir, img_name))
                        self.labels.append(class_idx)
        
        self.labels = np.array(self.labels)
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        image = Image.open(self.image_paths[idx]).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, self.labels[idx]

In [ ]:
# 변환 정의
train_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
# 데이터셋 로드
train_dir = os.path.join(DATA_DIR, 'train')
test_dir = os.path.join(DATA_DIR, 'test')

train_dataset = CatsDogsDataset(train_dir)
test_dataset = CatsDogsDataset(test_dir, transform=test_transform)

print(f"훈련: {len(train_dataset)}, 테스트: {len(test_dataset)}")
print(f"클래스 분포: cats={sum(train_dataset.labels==0)}, dogs={sum(train_dataset.labels==1)}")

In [ ]:
# 샘플 시각화
fig, axes = plt.subplots(2, 5, figsize=(12, 5))
temp_ds = CatsDogsDataset(train_dir, transform=test_transform)

for i, ax in enumerate(axes.flat):
    img, label = temp_ds[i * 50]
    img = img * torch.tensor([0.229, 0.224, 0.225]).view(3,1,1) + torch.tensor([0.485, 0.456, 0.406]).view(3,1,1)
    ax.imshow(img.permute(1,2,0).clip(0,1))
    ax.set_title('Cat' if label == 0 else 'Dog')
    ax.axis('off')
plt.tight_layout()
plt.show()

## 4. 모델 정의

### 망 구조 표기법
- **C**: Convolution (합성곱) - 공간적 특징 추출
- **BN**: Batch Normalization - 학습 안정화, 수렴 가속
- **R**: ReLU / **T**: Tanh - 활성화 함수
- **AvgP** / **MaxP**: Average/Max Pooling - 다운샘플링
- **GAP**: Global Average Pooling - 파라미터 감소
- **D**: Dropout - 정규화, 과적합 방지
- **FC**: Fully Connected - 분류를 위한 선형 변환
- **Flat**: Flatten - 2D→1D 변환
- **Res**: Residual Connection - skip connection
- **SE**: Squeeze-and-Excitation - 채널별 attention

---

### 4.1 LeNet-5 (베이스라인, LeCun et al., 1998)

**구조**: `C(3,6,5×5) → T → AvgP → C(6,16,5×5) → T → AvgP → Flat → FC → T → FC → T → FC`

**특징**:
- 최초의 성공적인 CNN (수업시간에 배운 모델)
- 5×5 큰 커널로 특징 추출
- RGB 입력에 맞게 변형 (원본은 grayscale)

**효과/한계**:
- ✅ CNN의 기본 원리 적용
- ❌ 128×128 큰 이미지에는 얕은 구조
- ❌ 현대적 기법(BN, ReLU, Dropout) 미적용
- ❌ 개/고양이 같은 복잡한 분류에 한계

In [ ]:
class LeNet5(nn.Module):
    """
    LeNet-5 (LeCun et al., 1998) - 128x128 RGB 버전
    구조: C(3,6,5x5) → T → AvgP → C(6,16,5x5) → T → AvgP → Flat → FC → T → FC → T → FC
    
    - 원본 LeNet-5를 RGB 입력에 맞게 변형
    - 5x5 커널과 Tanh 활성화 (원본 논문)
    - 128x128 → 62x62 → 31x31 → 13x13 → 6x6 (AvgPool 추가)
    """
    def __init__(self, num_classes=2):
        super().__init__()
        # 망 구조: C → T → AvgP → C → T → AvgP → C → T → AvgP → Flat → FC → T → FC → T → FC
        self.layers = nn.ModuleList([
            # Feature Extraction (128x128 입력 처리를 위해 레이어 추가)
            nn.Conv2d(3, 6, 5),                # C(3,6,5x5): 128 → 124
            nn.Tanh(),                          # T: 활성화
            nn.AvgPool2d(2, 2),                 # AvgP: 124 → 62
            nn.Conv2d(6, 16, 5),                # C(6,16,5x5): 62 → 58
            nn.Tanh(),                          # T: 활성화
            nn.AvgPool2d(2, 2),                 # AvgP: 58 → 29
            nn.Conv2d(16, 32, 5),               # C(16,32,5x5): 29 → 25
            nn.Tanh(),                          # T: 활성화
            nn.AvgPool2d(2, 2),                 # AvgP: 25 → 12
            nn.Flatten(),                       # Flat: (B,32,12,12) → (B,4608)
            # Classification
            nn.Linear(32 * 12 * 12, 120),       # FC(4608,120)
            nn.Tanh(),                          # T: 활성화
            nn.Linear(120, 84),                 # FC(120,84)
            nn.Tanh(),                          # T: 활성화
            nn.Linear(84, num_classes)          # FC(84,2): 출력층
        ])
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

print("LeNet-5: C(5x5) → T → AvgP → C(5x5) → T → AvgP → C(5x5) → T → AvgP → Flat → FC → T → FC → T → FC")
print(LeNet5())
print(f"파라미터: {sum(p.numel() for p in LeNet5().parameters()):,}")

### 4.2 VGGNet-style (Simonyan & Zisserman, 2014)

**구조**: `[C(3×3) → BN → R]×2 → MaxP → [C(3×3) → BN → R]×2 → MaxP → [C(3×3) → BN → R]×3 → MaxP → [C(3×3) → BN → R]×3 → GAP → D → FC`

**특징**:
- 작은 3×3 커널을 깊게 쌓음 (VGG의 핵심 아이디어)
- 3×3 두 번 = 5×5 수용 영역, 더 적은 파라미터
- Batch Normalization + ReLU로 현대화
- MaxPooling으로 점진적 다운샘플링

**효과**:
- ✅ 깊은 네트워크로 복잡한 패턴 학습
- ✅ 작은 커널로 파라미터 효율성
- ✅ BN으로 학습 안정화
- ⚠️ Skip connection 없어 gradient 문제 가능

In [ ]:
class VGGBlock(nn.Module):
    """
    VGG Block: 3x3 Conv를 여러 번 쌓은 블록
    구조: [C(3x3) → BN → R] × n_convs
    """
    def __init__(self, in_ch, out_ch, n_convs=2):
        super().__init__()
        layers = []
        for i in range(n_convs):
            layers.extend([
                nn.Conv2d(in_ch if i == 0 else out_ch, out_ch, 3, 1, 1),  # C(3x3)
                nn.BatchNorm2d(out_ch),                                   # BN
                nn.ReLU()                                                  # R
            ])
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        return x


class VGGNet(nn.Module):
    """
    VGGNet-style (Simonyan & Zisserman, 2014) - 128x128 RGB
    구조: VGGBlock×4 + MaxPool×4 + GAP + FC
    
    - 3x3 작은 커널만 사용 (VGG의 핵심)
    - BN과 ReLU로 현대화
    - 128x128 → 64 → 32 → 16 → 8 → GAP
    """
    def __init__(self, num_classes=2):
        super().__init__()
        # 망 구조: VGGBlock → MaxP → VGGBlock → MaxP → ... → GAP → D → FC
        self.layers = nn.ModuleList([
            # Block 1: 128x128 → 64x64
            VGGBlock(3, 32, n_convs=2),            # [C(3x3) → BN → R] × 2
            nn.MaxPool2d(2, 2),                     # MaxP
            # Block 2: 64x64 → 32x32
            VGGBlock(32, 64, n_convs=2),           # [C(3x3) → BN → R] × 2
            nn.MaxPool2d(2, 2),                     # MaxP
            # Block 3: 32x32 → 16x16
            VGGBlock(64, 128, n_convs=3),          # [C(3x3) → BN → R] × 3
            nn.MaxPool2d(2, 2),                     # MaxP
            # Block 4: 16x16 → 8x8
            VGGBlock(128, 256, n_convs=3),         # [C(3x3) → BN → R] × 3
            nn.MaxPool2d(2, 2),                     # MaxP: 8x8
            nn.AdaptiveAvgPool2d(1),               # GAP: 8x8 → 1x1
            nn.Dropout(0.5)                         # D: 과적합 방지
        ])
        self.fc = nn.Linear(256, num_classes)      # FC(256,2): 분류
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("VGGNet: [C(3x3)→BN→R]×2 → MaxP → ... (4 blocks) → GAP → D → FC")
print(VGGNet())
print(f"파라미터: {sum(p.numel() for p in VGGNet().parameters()):,}")

### 4.3 ResNetCNN (He et al., 2015)

**구조**: `C(3,32,7×7,s2) → BN → R → MaxP → Res(32,64) → Res(64,128) → Res(128,256) → GAP → D(0.5) → FC(256,2)`

**ResBlock 내부**: `C(3×3) → BN → R → C(3×3) → BN → (+shortcut) → R`

**특징**:
- Residual Connection (skip connection) 도입
- 7×7 큰 커널로 시작 (넓은 수용 영역)
- MaxPool로 초기 다운샘플링
- 3개의 Residual Block

**효과**:
- ✅ Skip connection으로 gradient flow 개선
- ✅ 깊은 네트워크에서도 안정적인 학습
- ✅ VGGNet보다 적은 파라미터로 더 좋은 성능
- ✅ 개/고양이 같은 복잡한 분류에 효과적

In [ ]:
class ResidualBlock(nn.Module):
    """
    Residual Block (He et al., 2015)
    구조: C → BN → R → C → BN → (+shortcut) → R
    
    핵심 아이디어: H(x) = F(x) + x
    - 잔차 F(x)를 학습하여 gradient flow 개선
    """
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(in_ch, out_ch, 3, stride, 1, bias=False),  # C: 3x3 conv
            nn.BatchNorm2d(out_ch),                              # BN
            nn.ReLU(),                                           # R
            nn.Conv2d(out_ch, out_ch, 3, 1, 1, bias=False),      # C: 3x3 conv
            nn.BatchNorm2d(out_ch)                               # BN
        ])
        self.shortcut = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )
    
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        out += self.shortcut(x)  # Residual: F(x) + x
        return nn.ReLU()(out)


class ResNetCNN(nn.Module):
    """
    ResNetCNN (He et al., 2015) - 128x128 RGB
    구조: C(7x7,s2) → BN → R → MaxP → Res(32,64) → Res(64,128) → Res(128,256) → GAP → D → FC
    
    - 7x7 큰 커널로 시작 (넓은 수용 영역)
    - 3개의 Residual Block
    """
    def __init__(self, num_classes=2):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(3, 32, 7, 2, 3, bias=False),   # C(3,32,7x7): 128 → 64
            nn.BatchNorm2d(32),                      # BN
            nn.ReLU(),                               # R
            nn.MaxPool2d(3, 2, 1),                   # MaxP: 64 → 32
            ResidualBlock(32, 64, 1),                # Res: 32x32
            ResidualBlock(64, 128, 2),               # Res: 16x16
            ResidualBlock(128, 256, 2),              # Res: 8x8
            nn.AdaptiveAvgPool2d(1),                 # GAP: 1x1
            nn.Dropout(0.5)                          # D
        ])
        self.fc = nn.Linear(256, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("ResNetCNN: C(7x7) → BN → R → MaxP → Res → Res → Res → GAP → D → FC")
print(ResNetCNN())
print(f"파라미터: {sum(p.numel() for p in ResNetCNN().parameters()):,}")

### 4.4 SEResNet (Hu et al., 2017) - 최고 성능 예상

**구조**: `C(3,32,7×7,s2) → BN → R → MaxP → SERes(32,64) → SERes(64,128) → SERes(128,256) → GAP → D(0.5) → FC(256,2)`

**SEResBlock 내부**: `C → BN → R → C → BN → SE → (+shortcut) → R`

**SE Block**: `GAP → FC(C,C/16) → R → FC(C/16,C) → Sigmoid → Scale`
- Squeeze: GAP로 채널별 전역 정보 압축
- Excitation: FC로 채널 간 관계 학습 후 중요도 계산

**특징**:
- Squeeze-and-Excitation 메커니즘
- 채널별 attention으로 털 패턴, 귀 모양 등 중요 특징 강조
- 수업에서 다루지 않은 최신 구조

**효과**:
- ✅ 채널별 중요도를 동적으로 조절
- ✅ 개/고양이 구분에 중요한 특징 강조
- ✅ 적은 파라미터 추가로 큰 성능 향상
- ✅ Residual + Attention 시너지

In [ ]:
class SEBlock(nn.Module):
    """
    Squeeze-and-Excitation Block (Hu et al., 2017)
    구조: GAP → FC(C, C/r) → R → FC(C/r, C) → Sigmoid → Scale
    """
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.AdaptiveAvgPool2d(1),                          # Squeeze
            nn.Flatten(),
            nn.Linear(channels, channels // reduction),       # FC: 축소
            nn.ReLU(),
            nn.Linear(channels // reduction, channels),       # FC: 복원
            nn.Sigmoid()
        ])
    
    def forward(self, x):
        b, c, _, _ = x.size()
        scale = x
        for layer in self.layers:
            scale = layer(scale)
        return x * scale.view(b, c, 1, 1)


class SEResidualBlock(nn.Module):
    """
    SE-Residual Block
    구조: C → BN → R → C → BN → SE → (+shortcut) → R
    """
    def __init__(self, in_ch, out_ch, stride=1):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(in_ch, out_ch, 3, stride, 1, bias=False),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(),
            nn.Conv2d(out_ch, out_ch, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_ch),
            SEBlock(out_ch)
        ])
        self.shortcut = nn.Sequential()
        if stride != 1 or in_ch != out_ch:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_ch, out_ch, 1, stride, bias=False),
                nn.BatchNorm2d(out_ch)
            )
    
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        out += self.shortcut(x)
        return nn.ReLU()(out)


class SEResNet(nn.Module):
    """
    SE-ResNet (Hu et al., 2017) - 128x128 RGB
    구조: C(7x7) → BN → R → MaxP → SERes×3 → GAP → D → FC
    """
    def __init__(self, num_classes=2):
        super().__init__()
        self.layers = nn.ModuleList([
            nn.Conv2d(3, 32, 7, 2, 3, bias=False),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(3, 2, 1),
            SEResidualBlock(32, 64, 1),
            SEResidualBlock(64, 128, 2),
            SEResidualBlock(128, 256, 2),
            nn.AdaptiveAvgPool2d(1),
            nn.Dropout(0.5)
        ])
        self.fc = nn.Linear(256, num_classes)
    
    def forward(self, x):
        for layer in self.layers:
            x = layer(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

print("SEResNet: C(7x7) → BN → R → MaxP → SERes → SERes → SERes → GAP → D → FC")
print(SEResNet())
print(f"파라미터: {sum(p.numel() for p in SEResNet().parameters()):,}")

## 5. 학습 함수

In [ ]:
def train_epoch(model, loader, criterion, optimizer):
    model.train()
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        loss = criterion(model(images), labels)
        loss.backward()
        optimizer.step()

def evaluate(model, loader):
    model.eval()
    preds, labels_list = [], []
    with torch.no_grad():
        for images, labels in loader:
            outputs = model(images.to(device))
            preds.extend(outputs.argmax(1).cpu().numpy())
            labels_list.extend(labels.numpy())
    return accuracy_score(labels_list, preds), f1_score(labels_list, preds, average='micro')

def train_model(model, train_loader, val_loader, epochs=20, lr=0.001):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=3)
    
    best_acc = 0
    best_state = None
    patience = 0
    
    for epoch in range(epochs):
        train_epoch(model, train_loader, criterion, optimizer)
        val_acc, _ = evaluate(model, val_loader)
        scheduler.step(val_acc)
        
        if val_acc > best_acc:
            best_acc = val_acc
            best_state = model.state_dict().copy()
            patience = 0
        else:
            patience += 1
            if patience >= 5:
                break
    
    if best_state:
        model.load_state_dict(best_state)
    return model

## 6. 반복 홀드아웃 검증 (5회)

In [ ]:
class TransformDataset(Dataset):
    def __init__(self, base_dataset, indices, transform):
        self.base = base_dataset
        self.indices = indices
        self.transform = transform
    
    def __len__(self):
        return len(self.indices)
    
    def __getitem__(self, idx):
        i = self.indices[idx]
        image = Image.open(self.base.image_paths[i]).convert('RGB')
        return self.transform(image), self.base.labels[i]

In [ ]:
def run_holdout(model_class, model_name, n_repeats=5):
    results = {'accuracy': [], 'f1_score': []}
    all_indices = np.arange(len(train_dataset))
    
    print(f"\n{'='*50}")
    print(f"{model_name}")
    print(f"{'='*50}")
    
    test_loader = DataLoader(test_dataset, batch_size=32)
    
    for i in range(n_repeats):
        set_seed(42 + i)
        
        # 3:2 분할
        train_idx, val_idx = train_test_split(
            all_indices, test_size=0.4, stratify=train_dataset.labels, random_state=42+i
        )
        
        train_subset = TransformDataset(train_dataset, train_idx, train_transform)
        val_subset = TransformDataset(train_dataset, val_idx, test_transform)
        
        train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
        val_loader = DataLoader(val_subset, batch_size=32)
        
        model = model_class().to(device)
        model = train_model(model, train_loader, val_loader, epochs=30)
        
        acc, f1 = evaluate(model, test_loader)
        results['accuracy'].append(acc)
        results['f1_score'].append(f1)
        print(f"[{i+1}/{n_repeats}] Acc: {acc:.4f}, F1: {f1:.4f}")
    
    return results

In [ ]:
lenet_results = run_holdout(LeNet5, "LeNet-5 (베이스라인)")

In [ ]:
vgg_results = run_holdout(VGGNet, "VGGNet-style")

In [ ]:
resnet_results = run_holdout(ResNetCNN, "ResNetCNN")

In [ ]:
senet_results = run_holdout(SEResNet, "SEResNet (최고 성능)")

## 7. 결과 분석

In [ ]:
def print_summary(results, name):
    acc_mean, acc_std = np.mean(results['accuracy']), np.std(results['accuracy'])
    f1_mean, f1_std = np.mean(results['f1_score']), np.std(results['f1_score'])
    print(f"{name}")
    print(f"  Accuracy: {acc_mean:.4f} ± {acc_std:.4f}")
    print(f"  F1 Score: {f1_mean:.4f} ± {f1_std:.4f}")
    return acc_mean, acc_std, f1_mean, f1_std

print("="*50)
print("최종 결과")
print("="*50)
lenet_stats = print_summary(lenet_results, "LeNet-5")
vgg_stats = print_summary(vgg_results, "VGGNet")
resnet_stats = print_summary(resnet_results, "ResNetCNN")
senet_stats = print_summary(senet_results, "SEResNet")

In [ ]:
# 시각화
fig, ax = plt.subplots(figsize=(12, 5))
x = np.arange(5)
width = 0.2

ax.bar(x - 1.5*width, lenet_results['accuracy'], width, label='LeNet-5')
ax.bar(x - 0.5*width, vgg_results['accuracy'], width, label='VGGNet')
ax.bar(x + 0.5*width, resnet_results['accuracy'], width, label='ResNetCNN')
ax.bar(x + 1.5*width, senet_results['accuracy'], width, label='SEResNet')

ax.set_xlabel('반복')
ax.set_ylabel('Accuracy')
ax.set_title('Cats vs Dogs - 모델 성능 비교')
ax.set_xticks(x)
ax.legend()
plt.tight_layout()
plt.show()

## 8. 결론

| 모델 | 구조 특징 | Accuracy | F1 Score |
|------|----------|----------|----------|
| LeNet-5 | C(5×5)-AvgP 반복, Tanh | - | - |
| VGGNet | [C(3×3)]×n-MaxP 반복 | - | - |
| ResNetCNN | Residual Connection | - | - |
| SEResNet | SE Attention + Residual | - | - |

### 모델별 분석

- **LeNet-5 (베이스라인)**: 1998년 고전적 구조. 5×5 큰 커널과 Tanh 활성화. 128×128 이미지에는 얕은 구조로 한계.

- **VGGNet-style**: 3×3 작은 커널을 깊게 쌓아 수용 영역 확보. BN과 ReLU로 현대화했으나 skip connection 없음.

- **ResNetCNN**: Residual Connection으로 gradient vanishing 해결. 입력을 출력에 더해 잔차 학습Jean.

- **SEResNet**: SE Block으로 채널별 중요도 동적 조절. 개/고양이 구분에서 털 패턴, 귀 모양 등 중요 특징 강조. 최고 성능 예상.